# Storing Google Places API data using GraphQL

This notebook documents the process to store data collected using **Google Places API**. 


### GQL client Interface

[GQL](https://github.com/graphql-python/gql), GraphQL client for Python is being used here to interface with GraphQL backend. **GraphQL** is used to store our JSON data into **MongoDB** for caching.

We intall GQL and create the client as shown below.

In [0]:
pip install gql

     |████████████████████████████████| 256kB 4.7MB/s 
     |████████████████████████████████| 184kB 43.9MB/s 
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5540 sha256=8c4717bd3bb7804c3f6bdab7c87e1d1634e522462cdcb39c4ef7d8607fc244b4
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
Successfully built gql


In [0]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
import json

expected_retries = 3
# URL information to be updated.
url='' 


client = Client(retries=expected_retries,
                transport=RequestsHTTPTransport(url=url))

### Data Insertion
The below code is to store static  place id and related details.

Note: Here we are not working with the complete Google Places API output.

In [0]:
# Hard coded value
# Checking how our API works
createBaseRSPlace = gql('''
mutation createBaseRSPlace {
  create_basers_place(place_data: {place_id: "ChIJN1t_tDeuEmsRUsoyG83frY4", google_place_details: "{\\"place_id\\":\\"ChIJN1t_tDeuEmsRUsoyG83frY4\\",\\"rating\\" : 4.6, \\"name\\" : \\"Google\\"}", timestamp: "2019-06-02T13:44:31+05:30"}) {
    place_id
    ok
    message
  }
}
''')

client.execute(createBaseRSPlace)

{'create_basers_place': {'message': 'Success',
  'ok': True,
  'place_id': 'ChIJN1t_tDeuEmsRUsoyG83frY4'}}

The below code shows how to store place id and related info dynamically into our database

In [0]:
# Dynamic variable
createBaseRSPlace = gql('''
mutation createBaseRSPlace($place_id: String!, 
                           $google_place_details: JSONString!,
                           $timestamp: CustomGrapheneDateTime!) {
  create_basers_place(place_data: {place_id: $place_id, 
                                   google_place_details: $google_place_details, 
                                   timestamp: $timestamp}) {
    place_id
    ok
    message
  }
}
''')

params = '''{
    "place_id": "ChIJN1t_tDeuEmsRUsoyG83frY4",
    "google_place_details": "{\\"place_id\\":\\"ChIJN1t_tDeuEmsRUsoyG83frY4\\",\\"rating\\" : 4.6, \\"name\\" : \\"Google\\"}",
    "timestamp": "2019-06-02T13:44:31+05:30"
}'''

client.execute(createBaseRSPlace, variable_values=params)

{'create_basers_place': {'message': 'Success',
  'ok': True,
  'place_id': 'ChIJN1t_tDeuEmsRUsoyG83frY4'}}

### Data Retrival: Checking Place details 
The code below checks how static place details are stored in our database.

In [0]:
# Hard coded value
# This again is hard coded where the variables aren't changing 
queryBaseRSPlaceDetails = gql('''
query queryBaseRSPlaceDetails {
  basers_place_details(place_id: "ChIJN1t_tDeuEmsRUsoyG83frY4") {
    place_id
    google_place_details
  }
}
''')

client.execute(queryBaseRSPlaceDetails)

{'basers_place_details': {'google_place_details': '{"place_id": "ChIJN1t_tDeuEmsRUsoyG83frY4", "rating": 4.6, "name": "Google"}',
  'place_id': 'ChIJN1t_tDeuEmsRUsoyG83frY4'}}

Dynamically retrieving the information from our database.

In [0]:
# Dynamic variable
queryBaseRSPlaceDetails = gql('''
query queryBaseRSPlaceDetails($place_id: String!) {
  basers_place_details(place_id: $place_id) {
    place_id
    google_place_details
  }
}
''')

params = '''{
    "place_id": "ChIJbf8C1yFxdDkR3n12P4DkKt0"
}'''

client.execute(queryBaseRSPlaceDetails, variable_values=params)

{'basers_place_details': {'google_place_details': '{"address_components": [{"long_name": "Dharmapuri", "short_name": "Dharmapuri", "types": ["neighborhood", "political"]}, {"long_name": "Forest Colony", "short_name": "Forest Colony", "types": ["sublocality_level_2", "sublocality", "political"]}, {"long_name": "Tajganj", "short_name": "Tajganj", "types": ["sublocality_level_1", "sublocality", "political"]}, {"long_name": "Agra", "short_name": "Agra", "types": ["locality", "political"]}, {"long_name": "Agra", "short_name": "Agra", "types": ["administrative_area_level_2", "political"]}, {"long_name": "Uttar Pradesh", "short_name": "UP", "types": ["administrative_area_level_1", "political"]}, {"long_name": "India", "short_name": "IN", "types": ["country", "political"]}, {"long_name": "282001", "short_name": "282001", "types": ["postal_code"]}], "adr_address": "<span class=\'extended-address\'>Dharmapuri, Forest Colony, Tajganj</span>, <span class=\'locality\'>Agra</span>, <span class=\'reg

### Data Retrival: Fetch All Place IDs

The code below retrieves all place IDs stored in our database. This shall be used for building our base recommendation system.

In [0]:
# check all place IDs
queryBaseRSAllPlaceIDs = gql('''
query queryBaseRSAllPlaceIDs {
  basers_all_place_ids{
    place_id
  }
}
''')

client.execute(queryBaseRSAllPlaceIDs)

{'basers_all_place_ids': [{'place_id': 'ChIJbf8C1yFxdDkR3n12P4DkKt0'},
  {'place_id': 'ChIJPTacEpBQwokRKwIlDXelxkA'},
  {'place_id': 'ChIJuU2C7F5E04kRiKK9VmHF0kY'},
  {'place_id': 'ChIJrwO1fgcmMxURNKNBOlKTnJw'},
  {'place_id': 'ChIJaXQRs6lZwokRY6EFpJnhNNE'},
  {'place_id': 'ChIJN1t_tDeuEmsRUsoyG83frY4'},
  {'place_id': 'ChIJw____96GhYARCVVwg5cT7c0'},
  {'place_id': 'ChIJ-aH5AxFwrzsRDdokoeK6f8M'},
  {'place_id': 'ChIJ42MF8e0VrjsRPoV7_ygSOhQ'},
  {'place_id': 'ChIJ2dGMjMMEdkgRqVqkuXQkj7c'},
  {'place_id': 'ChIJj2HZewpZ0IkRCRdRnhG9LNQ'},
  {'place_id': 'ChIJGymPrIdFWBQRJCSloj8vDIE'},
  {'place_id': 'ChIJzyx_aNch8TUR3yIFlZslQNA'},
  {'place_id': 'ChIJzYhOxKaR1RIRA_xU1bGp7DI'},
  {'place_id': 'ChIJuzIBtptoUjoRCrZi347PSQU'}]}

### Data Removal: Deleting entries from our database 
The below code is to delete a particular entry from our database

In [0]:
# Hard coded value
deleteBaseRSPlace = gql('''
mutation deleteBaseRSPlace {
  delete_basers_place(place_data: {place_id: "ChIJN1t_tDeuEmsRUsoyG83frY4"}) {
    place_id
    ok
    message
  }
}
''')
client.execute(deleteBaseRSPlace)

{'delete_basers_place': {'message': 'Success',
  'ok': True,
  'place_id': 'ChIJN1t_tDeuEmsRUsoyG83frY4'}}

Once the place is deleted, running the code again will result in the warning that "Delete from Mongo DB failed" because the same entry can't be deleted twice.

This code below shows dynamically passing variables for deleting from our database.

In [0]:
# Dynamic variable
deleteBaseRSPlace = gql('''
mutation deleteBaseRSPlace($place_id: String!) {
  delete_basers_place(place_data: {place_id: $place_id}) {
    place_id
    ok
    message
  }
}
''')

params = '''{
    "place_id": "ChIJw____96GhYARCVVwg5cT7c0"
}'''

client.execute(deleteBaseRSPlace, variable_values=params)



{'delete_basers_place': {'message': 'Delete from MongoDB failed',
  'ok': False,
  'place_id': 'ChIJw____96GhYARCVVwg5cT7c0'}}

# Working with Real Data

The below dictionary is the output for a particular Place ID that we retrived from Google Place API. Google Place API to search for any specific places e.g. **Statue or Liberty; Golden Gate Bridge...** Once you run that code, the output gives you place ID. 

We used that **Place ID** to query the below JSON result.

In [0]:
place_details = {'html_attributions': [],
 'result': {'address_components': [{'long_name': 'New York',
    'short_name': 'New York',
    'types': ['locality', 'political']},
   {'long_name': 'Manhattan',
    'short_name': 'Manhattan',
    'types': ['sublocality_level_1', 'sublocality', 'political']},
   {'long_name': 'New York County',
    'short_name': 'New York County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '10004', 'short_name': '10004', 'types': ['postal_code']}],
  'adr_address': '<span class="locality">New York</span>, <span class="region">NY</span> <span class="postal-code">10004</span>, <span class="country-name">USA</span>',
  'formatted_address': 'New York, NY 10004, USA',
  'formatted_phone_number': '(212) 363-3200',
  'geometry': {'location': {'lat': 40.6892494, 'lng': -74.04450039999999},
   'viewport': {'northeast': {'lat': 40.71814749999999,
     'lng': -73.99872490000001},
    'southwest': {'lat': 40.6796167, 'lng': -74.05975889999998}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
  'id': '5a0d7e67078e35af0c456a277df9ffba7c1e4da6',
  'international_phone_number': '+1 212-363-3200',
  'name': 'Statue of Liberty National Monument',
  'opening_hours': {'open_now': False,
   'periods': [{'close': {'day': 0, 'time': '1600'},
     'open': {'day': 0, 'time': '0830'}},
    {'close': {'day': 1, 'time': '1600'}, 'open': {'day': 1, 'time': '0830'}},
    {'close': {'day': 2, 'time': '1600'}, 'open': {'day': 2, 'time': '0830'}},
    {'close': {'day': 3, 'time': '1600'}, 'open': {'day': 3, 'time': '0830'}},
    {'close': {'day': 4, 'time': '1600'}, 'open': {'day': 4, 'time': '0830'}},
    {'close': {'day': 5, 'time': '1600'}, 'open': {'day': 5, 'time': '0830'}},
    {'close': {'day': 6, 'time': '1600'}, 'open': {'day': 6, 'time': '0830'}}],
   'weekday_text': ['Monday: 8:30 AM – 4:00 PM',
    'Tuesday: 8:30 AM – 4:00 PM',
    'Wednesday: 8:30 AM – 4:00 PM',
    'Thursday: 8:30 AM – 4:00 PM',
    'Friday: 8:30 AM – 4:00 PM',
    'Saturday: 8:30 AM – 4:00 PM',
    'Sunday: 8:30 AM – 4:00 PM']},
  'photos': [{'height': 5312,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/118296288152495182017/photos">miki nogal</a>'],
    'photo_reference': 'CmRaAAAA-r2MC6tSDKxHfYIuasA7QRLvrrR9pcBv46S7VKT9YC0mRy1_POzj2p_gAeMMV-udsdDh7sgAkri3ESf3LOR97eE_H6tpTxjFj5yF4hxk6zfs_G9yFy8pb5H4FsPllu2IEhCJz7N6JG0DuiY_QOVI5XPRGhRXSDhu2s5f9KOeb8BrN8ij2m6htg',
    'width': 2988},
   {'height': 4608,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106265366409122628933/photos">Isabel Castañeda</a>'],
    'photo_reference': 'CmRaAAAAMV7D_TqTKZfJ1hjXx9D89ifdPjPn7MmxunUn1Id1N8r3JoDaEeZt3CVB1-_H0oHZG5tWlNfNCheAL6M-pdxbajW6HJ4TVvnmmKjZAbYzza5KRcZn5PSWU_p8kELxtCvMEhBDIpRneg3eeiTT3JWSHiLcGhQAfqehQBwiZalBw3l5-D8wiNsM-w',
    'width': 3456},
   {'height': 2322,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112798933856821576176/photos">Jiasu BECERRA Y PACHECO</a>'],
    'photo_reference': 'CmRaAAAAwH3lqtHdmCm2Ki-fdoTlcLAgKqmr_NG-yWjWn4X5LZo2ZXnzwOM9oQ7nuAjgPQdvGBZ4ONZhsKQthrSuvr_r4EsB6QPeo3Zg6KqXAFyk69k_bNmbLeq3X2rSF0rWUBDhEhDly3R-IAt_KEVGF_NCwd04GhSpM62lQl_OEBv5EVtLSrRXfZOB0A',
    'width': 4128},
   {'height': 5184,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113176694194000645869/photos">Mark Donohoe</a>'],
    'photo_reference': 'CmRaAAAA5hIrcwUHN7Dp_xEoswbFH8mmfE3M1iiBJNORIx1tSfxKfwsOkZh9dD5HlCbuOzXwLC5nyUfhWTv8CUbpnxqaI5n60diEB533cgOT6xD9C_R_ibLZ6MptM-0JIohXGxwNEhB1Zk0WCr_Qh4hRX-ptiBQdGhSKE8svSlm8caZZ_bI71LPjovrSTw',
    'width': 3456},
   {'height': 4032,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114719983703531435092/photos">Marlene Houldsworth</a>'],
    'photo_reference': 'CmRaAAAAnU7GxdnXJ0eskGSrt-3bj_DpizKoXvi8mjQR0O3G7r2aZPHDX0LedI1okSHBiQUI8_Xn6wmSnhj7LVabFbLcaGmSajibzZ49UpypIcltQa2h7kYT4RuLa3DcuaMkJnlPEhDxLJjRGxoJlE4f0uEd4ro-GhTD5WidpscyVAm8CvIh4xlxPVaQmw',
    'width': 2268},
   {'height': 4032,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107405260379345079616/photos">sai kalvakota</a>'],
    'photo_reference': 'CmRaAAAA7EBrKx7ObSC-i1a7MkvRk-Ss9hyHCn7ejUOHL4S_rV2pJqOon7nlZjyHGnCwSHEy5e7-DetQyFJ9-oBlh56EoZ7RiPrXIkMiyQlNeJpG2BWVRYVajtxQDvBw60RIC90OEhAcZwhxOnxM4EmpunQCFl0hGhTo_ThX-mwSudprZFfYgIkR1XtBcg',
    'width': 1960},
   {'height': 4160,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110542286036073775446/photos">Jeffrey Peikin</a>'],
    'photo_reference': 'CmRaAAAAq2VQmqQtlF4KjzzbH15HlpzZTywntYNP-OhxjZs7glB1-VIhLJS3AjjN7GsTn_fXSvBvWGfMpg8DBx7LMghvW2rydk13ZStCCVjWuMJQjCRvyB3QqMH12m_XoCPu2ZZuEhAq3960wd83nwU2GYTctFy-GhQAFwod3c8Nuogurivr7A8vXAA_uw',
    'width': 2340},
   {'height': 2160,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117890961343677572763/photos">Armando Yanes Licea</a>'],
    'photo_reference': 'CmRaAAAA1DnN94wcC_m7jgxm80W_-1lGqSNI28GP7hrVuYqVHvoQyBxtVEF9NROpS2cLkuK9bpAJ1J04NLHDHUn4MJUP8LH4JcH7eQrE3RtQG-iefrNNKuD27Mpf_eGHjPunqiqlEhCfJH9DM-o6XXuqlbIr4s9gGhRej12V-QGig1yKKb21rGJkvE_mAg',
    'width': 2160},
   {'height': 4032,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109627772504829321801/photos">David Tobar</a>'],
    'photo_reference': 'CmRaAAAA_QpeyE9AY6voOcV0GatCO1MAScOw9Z8DCKhhHgmJ0gugy_kiVoZczB6j5X_fBe4NzChLOUeP9zyA_sN3wDZMlU8Va3UyxElzJbkIurTvX72_aqep3j7ERNGqKsUhKbjwEhDv9LiIXlpxJM_m14T7gIvsGhQ8eHTsKmXu2fsQA6vwzQB0er_ZBA',
    'width': 3024},
   {'height': 2304,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111177214989513874707/photos">Beny Jackson</a>'],
    'photo_reference': 'CmRaAAAAVOOSUO12_HLCSQdrWMg_wvSY9GtsMxRk-Gv_lo0aAdF9CjVuO1sBTOQ9Pnzvzw3NuxYId6tcvtK54JFMOjoywMX19t2Ja9iaYPJxxzA3waDEvgy5nVsBLTdTC4Xt05mVEhALcIs0LPAfz_2I7g4WjNYYGhRtNlJDC_TUHuPA2gnRupc23uJ_nA',
    'width': 3456}],
  'place_id': 'ChIJPTacEpBQwokRKwIlDXelxkA',
  'plus_code': {'compound_code': 'MXQ4+M5 New York, United States',
   'global_code': '87G7MXQ4+M5'},
  'rating': 4.6,
  'reference': 'ChIJPTacEpBQwokRKwIlDXelxkA',
  'scope': 'GOOGLE',
  'types': ['park', 'point_of_interest', 'establishment'],
  'url': 'https://maps.google.com/?cid=4667599994556318251',
  'user_ratings_total': 54426,
  'utc_offset': -240,
  'vicinity': 'New York',
  'website': 'https://www.nps.gov/stli/index.htm'},
 'status': 'OK'}

{"html_attributions": [], "result": {"address_components": [{"long_name": "New York", "short_name": "New York", "types": ["locality", "political"]}, {"long_name": "Manhattan", "short_name": "Manhattan", "types": ["sublocality_level_1", "sublocality", "political"]}, {"long_name": "New York County", "short_name": "New York County", "types": ["administrative_area_level_2", "political"]}, {"long_name": "New York", "short_name": "NY", "types": ["administrative_area_level_1", "political"]}, {"long_name": "United States", "short_name": "US", "types": ["country", "political"]}, {"long_name": "10004", "short_name": "10004", "types": ["postal_code"]}], "adr_address": "<span class=\"locality\">New York</span>, <span class=\"region\">NY</span> <span class=\"postal-code\">10004</span>, <span class=\"country-name\">USA</span>", "formatted_address": "New York, NY 10004, USA", "formatted_phone_number": "(212) 363-3200", "geometry": {"location": {"lat": 40.6892494, "lng": -74.04450039999999}, "viewport

### JSON Data Clean up

Before we can insert JSON data into MongoDB via GraphQL, it needs some cleaning up with respect to handling of special characters like ", ' and new line.

In [0]:
place_details_str = json.dumps(place_details)
print(place_details_str)
temp_place_details_str = place_details_str.replace('\\\"',"'")
print(temp_place_details_str)
updated_place_details_str = temp_place_details_str.replace('"', '\\"')
print(updated_place_details_str)

### Testing Insertion with Real Data

Using the above insertion API, let us try to store the actual Google Places API data.

In [0]:
# Dynamic variable
createBaseRSPlace = gql('''
mutation createBaseRSPlace($place_id: String!, 
                           $google_place_details: JSONString!,
                           $timestamp: CustomGrapheneDateTime!) {
  create_basers_place(place_data: {place_id: $place_id, 
                                   google_place_details: $google_place_details, 
                                   timestamp: $timestamp}) {
    place_id
    ok
    message
  }
}
''')

In [0]:
place_id = "ChIJN1t_tDeuEmsRUsoyG83frY4"
# google_place_details = "{\"place_id\":\"ChIJN1t_tDeuEmsRUsoyG83frY4\",\"rating\" : 4.6, \"name\" : \"Google\"}".replace('"', '\\"')
# google_place_details = "{\"place_id\":\"ChIJN1t_tDeuEmsRUsoyG83frY4\",\"rating\" : 4.6, \"attri\": [{\"name\" : \"Google\"}]}".replace('"', '\\"')
google_place_details = updated_place_details_str
timestamp = "2019-06-02T13:44:31+05:30"

place_id_str = f'    "place_id": "{place_id}",\n'
google_place_details_str = f'    "google_place_details": "{google_place_details}",\n'
timestamp_str = f'    "timestamp": "{timestamp}"\n'
params = "{\n"+place_id_str+google_place_details_str+timestamp_str+"}"
print(params)

{
    "place_id": "ChIJN1t_tDeuEmsRUsoyG83frY4",
    "google_place_details": "{\"html_attributions\": [], \"result\": {\"address_components\": [{\"long_name\": \"New York\", \"short_name\": \"New York\", \"types\": [\"locality\", \"political\"]}, {\"long_name\": \"Manhattan\", \"short_name\": \"Manhattan\", \"types\": [\"sublocality_level_1\", \"sublocality\", \"political\"]}, {\"long_name\": \"New York County\", \"short_name\": \"New York County\", \"types\": [\"administrative_area_level_2\", \"political\"]}, {\"long_name\": \"New York\", \"short_name\": \"NY\", \"types\": [\"administrative_area_level_1\", \"political\"]}, {\"long_name\": \"United States\", \"short_name\": \"US\", \"types\": [\"country\", \"political\"]}, {\"long_name\": \"10004\", \"short_name\": \"10004\", \"types\": [\"postal_code\"]}], \"adr_address\": \"<span class='locality'>New York</span>, <span class='region'>NY</span> <span class='postal-code'>10004</span>, <span class='country-name'>USA</span>\", \"formatte

In [0]:
client.execute(createBaseRSPlace, variable_values=params)

{'create_basers_place': {'message': 'Success',
  'ok': True,
  'place_id': 'ChIJN1t_tDeuEmsRUsoyG83frY4'}}

### Verifying the Insertion

Let us verify if the data was successfully stored by trying to fetch the details using previously mentioned APIs for retrieveing all Places IDs and specific Place Details.

Also to ensure the data is parsable, we shall use json.loads to check the data consistency.

In [0]:
client.execute(queryBaseRSAllPlaceIDs)

{'basers_all_place_ids': [{'place_id': 'ChIJbf8C1yFxdDkR3n12P4DkKt0'},
  {'place_id': 'ChIJPTacEpBQwokRKwIlDXelxkA'},
  {'place_id': 'ChIJuU2C7F5E04kRiKK9VmHF0kY'},
  {'place_id': 'ChIJrwO1fgcmMxURNKNBOlKTnJw'},
  {'place_id': 'ChIJaXQRs6lZwokRY6EFpJnhNNE'},
  {'place_id': 'ChIJN1t_tDeuEmsRUsoyG83frY4'}]}

In [0]:
response = client.execute(queryBaseRSPlaceDetails, variable_values=params)

In [0]:
json.loads(response['basers_place_details']['google_place_details'])

In [0]:
def get_place_details(place_id):
    # Dynamic variable
    queryBaseRSPlaceDetails = gql('''
    query queryBaseRSPlaceDetails($place_id: String!) {
      basers_place_details(place_id: $place_id) {
        place_id
        google_place_details
      }
    }
    ''')
    
    # Creating dynamic parameter list
    place_id_str = f'    "place_id": "{place_id}"\n'  # noqa: E999
    params = "{\n"+place_id_str+"}"

    return client.execute(queryBaseRSPlaceDetails, variable_values=params)

In [0]:
import pprint
pp = pprint.PrettyPrinter()

queryBaseRSAllPlaceIDs = gql('''
query queryBaseRSAllPlaceIDs {
  basers_all_place_ids{
    place_id
  }
}
''')

result = client.execute(queryBaseRSAllPlaceIDs)
all_place_info = result['basers_all_place_ids']

for place_info in all_place_info:
    place_id = place_info['place_id']
    print(f"Fetching place details for {place_id}...")
    place_details_result = get_place_details(place_id)
    place_details = place_details_result['basers_place_details']['google_place_details']
    pp.pprint(json.loads(place_details))

Fetching place details for ChIJbf8C1yFxdDkR3n12P4DkKt0...
{'address_components': [{'long_name': 'Dharmapuri',
                         'short_name': 'Dharmapuri',
                         'types': ['neighborhood', 'political']},
                        {'long_name': 'Forest Colony',
                         'short_name': 'Forest Colony',
                         'types': ['sublocality_level_2',
                                   'sublocality',
                                   'political']},
                        {'long_name': 'Tajganj',
                         'short_name': 'Tajganj',
                         'types': ['sublocality_level_1',
                                   'sublocality',
                                   'political']},
                        {'long_name': 'Agra',
                         'short_name': 'Agra',
                         'types': ['locality', 'political']},
                        {'long_name': 'Agra',
                         'short_name': 'Agra

# Time Formating

We shall use UTC time as reference within Experience Trail application and the format of Date Time stored is as shown below.

In [0]:
import pytz
import datetime

date = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
date = datetime.datetime(date.year, date.month, date.day,
                                     date.hour, date.minute, int(date.second),
                                     int(date.second * 1000000 % 1000000),
                                     tzinfo=date.tzinfo)


#'2019-08-01T10:43:30+00:00'
print(date.isoformat())

2019-08-01T14:46:40+00:00
